## 네이버 금융 가격 크롤링

### 삼성전자 긁어오기

In [1]:
from bs4 import BeautifulSoup
import requests as rq

url = 'https://finance.naver.com/item/sise.naver?code=005930'
res = rq.get(url)
html = res.text
print(html)


<!--  global include -->

	
	
<html lang='ko'>
<head>


	
		<title>삼성전자 : 네이버페이 증권</title>
	
	





	
	
		<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
	


<meta http-equiv="Content-Script-Type" content="text/javascript">
<meta http-equiv="Content-Style-Type" content="text/css">
<meta name="apple-mobile-web-app-title" content="네이버페이 증권" />





	
		<meta property="og:title" content="네이버페이 증권"/>
		<meta property="og:image" content="https://ssl.pstatic.net/static/m/stock/im/2016/08/og_stock-200.png"/>
		<meta property="og:url" content="https://finance.naver.com"/>
		<meta property="og:description" content="국내 해외 증시 지수, 시장지표, 뉴스, 증권사 리서치 등 제공"/>
    
    

<meta property="og:type" content="article"/>
<meta property="og:article:thumbnailUrl" content=""/>
<meta property="og:article:author" content="네이버페이 증권"/>
<meta property="og:article:author:url" content="http://FINANCE.NAVER.COM"/>






<link rel='stylesheet' type='text/css' href='https://ssl.pstatic.net/imgstock

### 가격

In [2]:
soup= BeautifulSoup(html, 'html.parser')
Money = soup.select_one("#_nowVal")
print(Money.text)

68,400


### 이름

In [3]:
Name = soup.find("div" , {'class':'wrap_company'})
Name=Name.find('a',{"href":"#"})
print(Name.text)

삼성전자


### 날짜

In [4]:
Date = soup.select_one("#time")
print(Date.text.strip())

2023.09.27 기준(장마감)


## 여러 회사를 긁어오기

2주차에서 말해주었지만, 크롤링을 할때, url의 변화를 봐야 된다고 말했는데, 주식 같은 경우는 번호에 항상 전용 주식 코드가 들어갑니다. 그렇기에 주식 코드를 스스로 알아내거나 파이썬 코드를 통해서 불러올 수도 있습니다.

In [5]:
!pip install -U finance-datareader

In [6]:
import FinanceDataReader as fdr
fdr.__version__

'0.9.50'

In [7]:
df_krx = fdr.StockListing('KRX')
display(df_krx)

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,68400,2,-200,-0.29,68600,69100,68200,14886491,1019873104916,408333126420000,5969782550,STK
1,373220,KR7373220003,LG에너지솔루션,KOSPI,,476500,1,1000,0.21,469500,476500,466500,256074,120706253500,111501000000000,234000000,STK
2,000660,KR7000660001,SK하이닉스,KOSPI,,114700,2,-300,-0.26,113000,115200,112300,3060751,348466084800,83501871265500,728002365,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,681000,1,1000,0.15,673000,686000,673000,57048,38781999000,48469494000000,71174000,STK
4,005490,KR7005490008,POSCO홀딩스,KOSPI,,535000,1,2000,0.38,528000,537000,525000,804112,427029573000,45245608050000,84571230,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2749,001525,KR7001521004,동양우,KOSPI,,5300,2,-160,-2.93,5260,5430,5260,6647,35308020,3273974300,617731,STK
2750,245450,KR7245450002,씨앤에스링크,KONEX,일반기업부,1598,2,-1,-0.06,1599,1599,1400,4,5997,2524776080,1579960,KNX
2751,288490,KR7288490006,나라소프트,KONEX,일반기업부,55,1,4,7.84,53,55,50,67959,3587092,2379685275,43267005,KNX
2752,308700,KR7308700004,테크엔,KONEX,일반기업부,505,1,19,3.91,485,555,480,166,86335,2020000000,4000000,KNX


In [8]:
import pandas as pd

date_list=[]
money_list=[]
name_list=[]
for i in df_krx['Code'][:10]:
    url = 'https://finance.naver.com/item/sise.naver?code='+str(i)
    res = rq.get(url)
    html = res.text
    soup= BeautifulSoup(html, 'html.parser')
    Money = soup.select_one("#_nowVal").text
    Date = soup.select_one("#time").text.strip()
    Name = soup.find("div" , {'class':'wrap_company'})
    Name=Name.find('a',{"href":"#"}).text
    date_list.append(Date)
    money_list.append(Money)
    name_list.append(Name)


Jusik=pd.DataFrame({'날짜':date_list, '시가':money_list},index=name_list)
display(Jusik)


,날짜,시가
삼성전자,2023.09.27 기준(장마감),"68,400"
LG에너지솔루션,2023.09.27 기준(장마감),"476,500"
SK하이닉스,2023.09.27 기준(장마감),"114,700"
삼성바이오로직스,2023.09.27 기준(장마감),"681,000"
POSCO홀딩스,2023.09.27 기준(장마감),"535,000"
삼성전자우,2023.09.27 기준(장마감),"54,500"
현대차,2023.09.27 기준(장마감),"191,100"
삼성SDI,2023.09.27 기준(장마감),"512,000"
LG화학,2023.09.27 기준(장마감),"496,500"
NAVER,2023.09.27 기준(장마감),"201,500"
